In [1]:
import pandas as pd
import glob
import time
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import datetime
import os
pd.options.display.max_rows = 6
pd.options.display.max_columns = 200

In [2]:
#all input file path
rootmount = "/mnt/adls"
_40_project = rootmount+"/40_project"
PrmPrdMount = _40_project+"/AAF/Alerting_Flow_PC/"
#parallelFlow = _40_project+"/MAD/AGB/ParallelFlow"
parallelFlow = _40_project+"/AAF/Alerting_Flow_PC/"
_20_datastore_D2 = rootmount+"/20_datastore_D2/"
_20_datastore = rootmount+"/20_datastore/"
RAG = _20_datastore+"/RAG"
CT2 = _20_datastore+"/CT2"
_outsourcing = _20_datastore+"/Outsourcing"
readCSV = spark.read.format("csv").options(header='true', delimiter = ';',skipinitialspace=True)

In [3]:
#load fact table
from pyspark.sql import functions as F
from pyspark.sql.types import *
fact_load = spark.read.format("csv").options(header='true', delimiter = #';').load(_40_project + "/MAD/Sales/Sales_Month/FDM_SALES_WW_FACT.csv")
',').load(rootmount + "/40_project/AAF/Alerting_Flow_PC/Input/Alert_flow.csv")                                          
# ';').load(rootmount + "/30_datamart_D2/MAD/Sales/FDM_SALES_WW_FACT.csv")                                          
fact_load.registerTempTable("fact_load")

# display(sql("SELECT * FROM fact_load "))

#fact = sql("select * From fact_load where LB_CODE = 'ARB'")
fact = sql("select * From fact_load where LB_CODE = 'AGB'")
fact.registerTempTable("fact")

#display(sql("SELECT * FROM fact "))
# fact1 = readCSV.load(_40_project + "/MAD/Cat_Commercial/CatCom_complet_avec_CAI_RAG.csv")
# fact1.registerTempTable("fact1")

fact_price = readCSV.load(_20_datastore_D2 + "Outsourcing/Manual/Corridor/FDM_CORRIDOR_COUNTRY_FACT.csv")
fact_price.registerTempTable("fact_price")

fact_final = sql("select distinct a.* ,b.MIN_PRICE_AMT as mini,b.MAX_PRICE_AMT as max from fact a left join fact_price b  on CAST(a.CAI_CODE as int) = CAST(b.CAI_CODE as int) and CAST(a.COUNTRY_CODE AS int) = CAST(b.COUNTRY_CODE AS int) and a.DT_YEARMONTH =b.DT_YEARMONTH ")
fact_final.registerTempTable("fact_final")

brand_load = readCSV.load(RAG + "/Product/FDM_BRAND_CMN_DIM.csv")
brand_load.registerTempTable("brand_load")



country = readCSV.load(CT2 + "/Geography/FDM_COUNTRY_CMN_DIM.csv")
country.registerTempTable("country_df")

region_country = readCSV.load(CT2 + "/Geography/FDM_REF_GEO_HIE_CMN_DIM.csv")
region_country.registerTempTable("region_countrydf")

region = readCSV.load(CT2 + "/Geography/FDM_REGION_CMN_DIM.csv")
region.registerTempTable("regiondf")
#display(sql("select * from country_df"))
#cust pricing
cust_pricing = readCSV.load(rootmount + "/30_datamart/KBI/Referential/FDM_CUST_PRICING_CMN_DIM.csv")
cust_pricing.registerTempTable("custpricing_df")
#cust level
cust_hie = readCSV.load(_outsourcing + "/Manual/FDM_CUST_LVL_HIE_CMN_DIM.csv")
cust_lvl0 = readCSV.load(_outsourcing + "/Manual/FDM_CUST_LVL0_HIE_CMN_DIM.csv")
cust_lvl1 = readCSV.load(_outsourcing + "/Manual/FDM_CUST_LVL1_HIE_CMN_DIM.csv")
cust_lvl2 = readCSV.load(_outsourcing + "/Manual/FDM_CUST_LVL2_HIE_CMN_DIM.csv")
cust_lvl3 = readCSV.load(_outsourcing + "/Manual/FDM_CUST_LVL3_HIE_CMN_DIM.csv")

cust_hie.registerTempTable("cust_hiedf")
cust_lvl0.registerTempTable("cust_lvl0")
cust_lvl1.registerTempTable("cust_lvl1")
cust_lvl2.registerTempTable("cust_lvl2")
cust_lvl3.registerTempTable("cust_lvl3")

#seat dia and season
cai_seas = readCSV.load(RAG + "/Product/FDM_SEASON_S_W_CMN_DIM.csv")
cai_seas.registerTempTable("cai_seas")
cai_mark = readCSV.load(RAG + "/Product/FDM_CAI_MARKET_ATTR_CMN_DIM.csv")
cai_mark.registerTempTable("cai_mark")


# b2b_sp = spark.read.format("csv").options(header='true', delimiter = ',').load(_40_project + "/AGA/B2B_PC/Outfile/B2B_alert_PC2.csv")
b2b_sp = spark.read.format("csv").options(header='true', delimiter = ' ').load(_40_project + "/AAF/Alerting_Flow_PC/B2B_PC/B2B_alert_PC2.csv")
b2b_sp.registerTempTable("b2b_sp")


#dfi = spark.read.format("csv").options(header='true', delimiter = ',').load("/FileStore/tables/WW_Successors__updated-74382.csv")
dfi = spark.read.format("csv").options(header='true', delimiter = ',').load(_40_project + "/AAF/Alerting_Flow_PC/Input/WW_SUCCESOR/WWW_SUCCESOR_2303.csv")
dfi.registerTempTable("dfi")
#b2b_sp = spark.read.format("csv").options(header='true', delimiter = ',').load("/FileStore/tables/B2B_alert_updated.csv")
#b2b_sp.registerTempTable("b2b_sp")

cdi_desc = readCSV.load(RAG + "/Product/FDM_CAI_ATTR_DIM.csv")
cdi_desc.registerTempTable("cdi_desc")



brand_load = readCSV.load(RAG + "/Product/FDM_BRAND_CMN_DIM.csv")
brand_load.registerTempTable("brand_load")

